In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging

In [2]:
# Utilizando raw string para la ruta del archivo
archivo_csv = r"C:\Users\JSLV3\Documents\5to Semestre\ETL\Proyecto Water Quality\1ra parte\watersucia.csv"

# Cargar los datos desde el archivo CSV
water = pd.read_csv(archivo_csv, delimiter=';')

# Mostrar las primeras filas del DataFrame
print(water.head())



    Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni IrcaMinimo  \
0  2010            Bolívar        13        El Guamo      13248          0   
1  2010            Bolívar        13        El Guamo      13248          0   
2  2010            Bolívar        13        El Guamo      13248          0   
3  2010            Bolívar        13        El Guamo      13248          0   
4  2010            Bolívar        13        El Guamo      13248          0   

  IrcaMaximo IrcaPromedio NombreParametroAnalisis2  MuestrasEvaluadas  \
0        100        37,32        Alcanilidad Total                 67   
1        100        37,32                 Aluminio                 67   
2        100        37,32                 Arsénico                 67   
3        100        37,32                   Cadmio                 67   
4        100        37,32                   Calcio                 67   

   MuestrasTratadas  MuestrasSinTratar  NumeroParametrosMinimo  \
0                67       

## Creating greater value from our data
### Transformations

##### We identified the analysis parameters that have the greatest influence on water pollution:

Conversión de Tipos de Datos

In [3]:
# Convertir las columnas IrcaMinimo, IrcaMaximo, y IrcaPromedio a flotante
water['IrcaMinimo'] = water['IrcaMinimo'].str.replace(',', '.').astype(float)
water['IrcaMaximo'] = water['IrcaMaximo'].str.replace(',', '.').astype(float)
water['IrcaPromedio'] = water['IrcaPromedio'].str.replace(',', '.').astype(float)

# Mostrar la información actualizada del dataset para confirmar los cambios
water[['IrcaMinimo', 'IrcaMaximo', 'IrcaPromedio']].info(), water[['IrcaMinimo', 'IrcaMaximo', 'IrcaPromedio']].head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408311 entries, 0 to 408310
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   IrcaMinimo    408311 non-null  float64
 1   IrcaMaximo    408311 non-null  float64
 2   IrcaPromedio  408311 non-null  float64
dtypes: float64(3)
memory usage: 9.3 MB


(None,
    IrcaMinimo  IrcaMaximo  IrcaPromedio
 0         0.0       100.0         37.32
 1         0.0       100.0         37.32
 2         0.0       100.0         37.32
 3         0.0       100.0         37.32
 4         0.0       100.0         37.32)

IrcaMinimo, IrcaMaximo, IrcaPromedio: Estas columnas contienen valores numéricos que estaban en formato de texto y utilizaban comas como separadores decimales. La conversión a flotante es crucial porque permite realizar cálculos numéricos adecuados, que son esenciales para cualquier análisis estadístico, comparaciones o visualizaciones que impliquen estos valores.

Normalización de Datos

In [4]:
scaler = MinMaxScaler()
columns_to_scale = ['MuestrasEvaluadas', 'MuestrasTratadas', 'MuestrasSinTratar']

water[columns_to_scale] = scaler.fit_transform(water[columns_to_scale])

water[columns_to_scale].head()

NameError: name 'MinMaxScaler' is not defined

MuestrasEvaluadas, MuestrasTratadas, MuestrasSinTratar: La normalización a una escala de 0 a 1 de estas columnas es fundamental para preparar los datos para algoritmos de aprendizaje automático y para facilitar comparaciones directas entre variables que originalmente tienen diferentes escalas y rangos. Por ejemplo, si una variable típicamente varía entre 0 y 1000 y otra entre 0 y 1000000, la normalización permite que ambas contribuyan equitativamente a los análisis sin que una domine debido a su mayor magnitud.

In [5]:
def standardize_column_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    logging.info("Column names standardized")
    return df

def clean_text_data(df):
    """ Limpia y estandariza texto en columnas específicas. """
    df['nombredepartamento'] = df['nombredepartamento'].str.capitalize()  # Primera letra en mayúscula, el resto en minúscula
    df['nombremunicipio'] = df['nombremunicipio'].str.capitalize()
    return df




In [6]:
parametros_influencia = water.groupby('NombreParametroAnalisis2')['IrcaPromedio'].mean().sort_values(ascending=False)
top_15_parametros = parametros_influencia.head(15)
top_15_parametros

NombreParametroAnalisis2
ph                               23.941989
Cromo total                      23.941989
Olor                             23.941989
Mesófilos                        23.941989
Mercurio                         23.941989
Manganeso                        23.941989
Magnesio                         23.941989
Hierro total                     23.941989
Organofosforados y carbamatos    23.941989
Alcanilidad Total                23.941989
Fosfatos                         23.941989
Fluoruros                        23.941989
Plomo                            23.941989
E.coli                           23.941989
Dureza total                     23.941989
Name: IrcaPromedio, dtype: float64

All these parameters have an average IRCA of approximately 23.94, suggesting an association with a considerable risk level. This can be useful for prioritizing which water quality parameters need more critical attention in monitoring and treatment programs.

In [7]:
water = water[water['NombreParametroAnalisis2'].isin(top_15_parametros.index)]

water.head(), water.shape

(     Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni  IrcaMinimo  \
 0   2010            Bolívar        13        El Guamo      13248         0.0   
 13  2010            Bolívar        13        El Guamo      13248         0.0   
 14  2010            Bolívar        13        El Guamo      13248         0.0   
 15  2010            Bolívar        13        El Guamo      13248         0.0   
 16  2010            Bolívar        13        El Guamo      13248         0.0   
 
     IrcaMaximo  IrcaPromedio NombreParametroAnalisis2  MuestrasEvaluadas  \
 0        100.0         37.32        Alcanilidad Total                 67   
 13       100.0         37.32              Cromo total                 67   
 14       100.0         37.32             Dureza total                 67   
 15       100.0         37.32                   E.coli                 67   
 16       100.0         37.32                Fluoruros                 67   
 
     MuestrasTratadas  MuestrasSinTratar  Numero

We have filtered the dataset to only leave the rows corresponding to the top 15 analysis parameters related to water pollution, and the rest have been removed.

In [12]:
max_parametros = water['NumeroParametrosMaximo'].max()

water['cobertura_analisis'] = (
    (water['NumeroParametrosPromedio'] / max_parametros) *
    (1 - ((water['NumeroParametrosMaximo'] - water['NumeroParametrosMinimo']) / max_parametros))
)

water[['cobertura_analisis']].head()

,cobertura_analisis
0,0.045706
13,0.045706
14,0.045706
15,0.045706
16,0.045706


##### Classification of the Water Quality Risk Index (IRCA)

In [13]:
def clasificar_irca(irca):
    try:
        # Si irca ya es numérico (float), no se intenta reemplazar comas
        if not isinstance(irca, float):
            irca = float(irca.replace(',', '.'))
        if irca == 0:
            return 'Sin información'
        elif 0.001 <= irca <= 5:
            return 'Sin riesgo'
        elif 5.001 <= irca <= 14:
            return 'Riesgo bajo'
        elif 14.001 <= irca <= 35:
            return 'Riesgo medio'
        elif 35.001 <= irca <= 80:
            return 'Riesgo alto'
        elif 80.001 <= irca <= 100:
            return 'Inviable sanitariamente'
        else:
            return 'No clasificado'
    except ValueError:
        return 'No clasificado'

water['rango_irca'] = water['IrcaPromedio'].apply(clasificar_irca)
print(water[['rango_irca','IrcaPromedio']].head())


     rango_irca  IrcaPromedio
0   Riesgo alto         37.32
13  Riesgo alto         37.32
14  Riesgo alto         37.32
15  Riesgo alto         37.32
16  Riesgo alto         37.32



The classify_irca function transforms the numerical value of the Average IRCA into descriptive categories ranging from 'No risk' to 'Sanitarily unviable', facilitating the interpretation and decision-making in water quality management. This categorization is crucial for public health and environmental analyses, as it simplifies data visualization, allows for quick comparisons between regions, and is essential for modeling and predicting water quality, resulting in more effective interventions and evidence-based policies.

In [14]:
water['porcentaje_muestras_tratadas'] = (water['MuestrasTratadas'] / water['MuestrasEvaluadas']) * 100

# Mostrar las primeras filas para confirmar la creación de la columna
print(water.head())


     Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni  IrcaMinimo  \
0   2010            Bolívar        13        El Guamo      13248         0.0   
13  2010            Bolívar        13        El Guamo      13248         0.0   
14  2010            Bolívar        13        El Guamo      13248         0.0   
15  2010            Bolívar        13        El Guamo      13248         0.0   
16  2010            Bolívar        13        El Guamo      13248         0.0   

    IrcaMaximo  IrcaPromedio NombreParametroAnalisis2  MuestrasEvaluadas  ...  \
0        100.0         37.32        Alcanilidad Total                 67  ...   
13       100.0         37.32              Cromo total                 67  ...   
14       100.0         37.32             Dureza total                 67  ...   
15       100.0         37.32                   E.coli                 67  ...   
16       100.0         37.32                Fluoruros                 67  ...   

    NumeroParametrosMinimo  Nume

Esta métrica cuantifica el éxito de los esfuerzos de tratamiento al revelar qué porción de las muestras de agua evaluadas han sido efectivamente tratadas. Un porcentaje elevado refleja una operación de tratamiento efectiva y una cobertura amplia, aspectos cruciales para la salud pública. Por otro lado, un porcentaje más bajo puede señalar oportunidades de mejora en la infraestructura o en las operaciones de tratamiento. 

Amplitud de Evaluación de Calidad del Agua:

In [15]:
water['rango_parametros_analizados'] = water['NumeroParametrosMaximo'] - water['NumeroParametrosPromedio']

print(water[['NumeroParametrosMaximo', 'NumeroParametrosPromedio', 'rango_parametros_analizados']].head())


    NumeroParametrosMaximo  NumeroParametrosPromedio  \
0                        7                         2   
13                       7                         2   
14                       7                         2   
15                       7                         2   
16                       7                         2   

    rango_parametros_analizados  
0                             5  
13                            5  
14                            5  
15                            5  
16                            5  


Esta columna se deriva de la diferencia entre el número máximo posible de parámetros que se podrían examinar y el número promedio de parámetros que efectivamente se han analizado en las pruebas de calidad del agua. Al calcular esta diferencia, obtenemos una perspectiva directa sobre la amplitud de las pruebas realizadas.

##### Treatment Category

In [16]:
def categorize_treatment(row):
    if row['MuestrasTratadas'] == 0:
        return 'Sin tratamiento'
    elif row['MuestrasTratadas'] == row['MuestrasEvaluadas']:
        return 'Tratamiento completo'
    else:
        return 'Tratamiento parcial'

water['TratamientoCategoría'] = water.apply(categorize_treatment, axis=1)
water.head()


,Año,NombreDepartamento,Div_dpto,NombreMunicipio,Divi_muni,IrcaMinimo,IrcaMaximo,IrcaPromedio,NombreParametroAnalisis2,MuestrasEvaluadas,...,NumeroParametrosMaximo,NumeroParametrosPromedio,ResultadoMinimo,ResultadoMaximo,ResultadoPromedio,cobertura_analisis,rango_irca,porcentaje_muestras_tratadas,rango_parametros_analizados,TratamientoCategoría
0,2010,Bolívar,13,El Guamo,13248,0.0,100.0,37.32,Alcanilidad Total,67,...,7,2,23,23,23,0.045706,Riesgo alto,100.0,5,Tratamiento completo
13,2010,Bolívar,13,El Guamo,13248,0.0,100.0,37.32,Cromo total,67,...,7,2,NaN,NaN,NaN,0.045706,Riesgo alto,100.0,5,Tratamiento completo
14,2010,Bolívar,13,El Guamo,13248,0.0,100.0,37.32,Dureza total,67,...,7,2,68,68,68,0.045706,Riesgo alto,100.0,5,Tratamiento completo
15,2010,Bolívar,13,El Guamo,13248,0.0,100.0,37.32,E.coli,67,...,7,2,0,1,"0,13",0.045706,Riesgo alto,100.0,5,Tratamiento completo
16,2010,Bolívar,13,El Guamo,13248,0.0,100.0,37.32,Fluoruros,67,...,7,2,NaN,NaN,NaN,0.045706,Riesgo alto,100.0,5,Tratamiento completo



The "Treatment Category" column classifies each set of water samples according to the degree of treatment they have received. This classification helps understand the management and effectiveness of treatment processes implemented in different locations. The categories are:

No treatment: Indicates that none of the evaluated samples were treated.

Partial treatment: Indicates that a portion of the evaluated samples was treated, but not all.

Complete treatment: Indicates that all evaluated samples were treated.

##### Data Cleaning and Variable Selection

In [17]:
water = water.drop(['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio'], axis=1)


To obtain a deeper understanding of the methodology and justification behind the removal of the 'Minimum Result', 'Maximum Result', and 'Average Result' columns, we invite you to consult the EDA_water_quality file. This document contains our exploratory data analysis, which distills key criteria and reveals significant insights that have guided the cleaning of our dataset.

In [20]:
columnas_a_eliminar = ['MuestrasTratadas', 'MuestrasEvaluadas', 'MuestrasSinTratar',
                      'NumeroParametrosMinimo', 'NumeroParametrosMaximo']
water = water.drop(columns=columnas_a_eliminar)


water.head()

KeyError: "['MuestrasTratadas', 'MuestrasEvaluadas', 'MuestrasSinTratar', 'NumeroParametrosMinimo', 'NumeroParametrosMaximo'] not found in axis"

In [19]:
new_column_names = [
    "año", 
    "nombre_departamento", 
    "div_dpto", 
    "nombre_municipio", 
    "divi_muni", 
    "irca_minimo", 
    "irca_maximo", 
    "irca_promedio", 
    "nombre_parametro_analisis", 
    "numero_parametros_promedio", 
    "cobertura_analisis",
    "rango_irca",
    "tratamiento_categoría"
    "rango_parametros_analizados",
    "porcentaje_muestras_tratadas"
    
    
]
water.columns = new_column_names
print(water.head())

ValueError: Length mismatch: Expected axis has 15 elements, new values have 14 elements

In [37]:
water.to_csv('water_cleaned.csv', index=False)